<a href="https://colab.research.google.com/github/scaltintasli/Airport-Runway-FOD/blob/main/fodTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/scaltintasli/Airport-Runway-FOD.git

Cloning into 'Airport-Runway-FOD'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 157 (delta 70), reused 108 (delta 38), pack-reused 0
Receiving objects: 100% (157/157), 43.81 MiB | 31.81 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [2]:
! cd Airport-Runway-FOD/Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 65017, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 65017 (delta 13), reused 21 (delta 8), pack-reused 64983
Receiving objects: 100% (65017/65017), 575.26 MiB | 32.00 MiB/s, done.
Resolving deltas: 100% (45513/45513), done.


In [4]:
!cd Airport-Runway-FOD/Tensorflow/scripts && git clone https://github.com/nicknochnack/GenerateTFRecord

Cloning into 'GenerateTFRecord'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [26]:
!cd Airport-Runway-FOD/Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && python object_detection/packages/tf2/setup.py build && python object_detection/packages/tf2/setup.py install

running build
running build_py
copying object_detection/protos/faster_rcnn_box_coder_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/fpn_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/preprocessor_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/box_predictor_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/post_processing_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/target_assigner_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/calibration_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/image_resizer_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/mean_stddev_box_coder_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/region_similarity_calculator_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/model_pb2.py ->

In [ ]:
!pip install tensorflow-object-detection-api

In [29]:
import os
import object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [30]:
CUSTOM_MODEL_NAME = "my_ssd_mobnet"
PRETRAINED_MODEL_NAME = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
TF_RECORD_SCRIPT_NAME = "generate_tfrecord.py"
LABEL_MAP_NAME = "label_map.pbtxt"

In [31]:
paths = {
    "WORKSPACE_PATH": os.path.join("Airport-Runway-FOD", "Tensorflow", "workspace"),
    "SCRIPTS_PATH": os.path.join("Airport-Runway-FOD", "Tensorflow", "scripts"),
    "APIMODEL_PATH": os.path.join("Airport-Runway-FOD", "Tensorflow", "models"),
    "ANNOTATION_PATH": os.path.join("Airport-Runway-FOD", "Tensorflow", "workspace", "annotations"),
    "IMAGE_PATH": os.path.join("Airport-Runway-FOD", "Tensorflow", "workspace", "images"),
    "MODEL_PATH": os.path.join("Airport-Runway-FOD", "Tensorflow", "workspace", "models"),
    "PRETRAINED_MODEL_PATH": os.path.join(
        "Airport-Runway-FOD", "Tensorflow", "workspace", "pre-trained-models"
    ),
    "CHECKPOINT_PATH": os.path.join(
        "Airport-Runway-FOD", "Tensorflow", "workspace", "models", CUSTOM_MODEL_NAME
    ),
    "OUTPUT_PATH": os.path.join(
        "Airport-Runway-FOD", "Tensorflow", "workspace", "models", CUSTOM_MODEL_NAME, "export"
    ),
    "TFJS_PATH": os.path.join(
        "Airport-Runway-FOD", "Tensorflow", "workspace", "models", CUSTOM_MODEL_NAME, "tfjsexport"
    ),
    "TFLITE_PATH": os.path.join(
        "Airport-Runway-FOD", "Tensorflow", "workspace", "models", CUSTOM_MODEL_NAME, "tfliteexport"
    ),
    "PROTOC_PATH": os.path.join("Airport-Runway-FOD", "Tensorflow", "protoc"),
}

In [32]:

files = {
    "PIPELINE_CONFIG": os.path.join(
        "Airport-Runway-FOD", "Tensorflow", "workspace", "models", CUSTOM_MODEL_NAME, "pipeline.config"
    ),
    "TF_RECORD_SCRIPT": os.path.join(paths["SCRIPTS_PATH"], TF_RECORD_SCRIPT_NAME),
    "LABELMAP": os.path.join(paths["ANNOTATION_PATH"], LABEL_MAP_NAME),
}


In [33]:
##List the items that needs to be detect
labels = [{"name": "screw", "id": 1}]
##Create A label map
with open(files["LABELMAP"], "w") as f:
    for label in labels:
        f.write("item { \n")
        f.write("\tname:'{}'\n".format(label["name"]))
        f.write("\tid:{}\n".format(label["id"]))
        f.write("}\n")

In [35]:
## This parts updates config file according to labels list length and paths
print("Updating Config File")
config = config_util.get_configs_from_pipeline_file(files["PIPELINE_CONFIG"])
config

Updating Config File


AttributeError: ignored